# Upgrade do artigo original

A ideia aqui é:
* salvar numpy arrays e usar elas diretamente no tensorflow, ao inves de converter em imagem
* diminuir o batch size
* usar um timeframe maior pra pagar menos taxas nas operações

In [1]:
import datetime as dt
import wget
import os
import pandas as pd
import numpy as np
#from zipfile import ZipFile
import zipfile
from timeit import default_timer as timer
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField
from keras.utils import load_img
from keras.utils import save_img
from keras.utils import img_to_array
from keras.utils import array_to_img
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.optimizers import Adagrad
from keras.optimizers import SGD
import funcoes as f
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # or any {'0', '1', '2'}
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import plotly.express as px
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from joblib import dump, load
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import absl.logging
from tqdm.auto import tqdm

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

absl.logging.set_verbosity(absl.logging.ERROR)
tf.get_logger().setLevel('WARNING')
tf.config.run_functions_eagerly(True)
#tf.keras.mixed_precision.set_global_policy("mixed_float16")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# define o ticker do ativo que vou usar pra treinar o modelo
ticker = "BTCUSDT"
# timeframes precisa estar em ordem crescente e começar em 1s ou 1m
timeframes = ("1m", "5m", "15m", "30m", "1h")#, "2h", "4h", "8h", "1d")
# quantos períodos vamos olhar pro passado
lookback = 20
# numero de quantis pra usar no markov transition field. Precisa tunar.
quantis = 3
# pega número de timeframes por minuto ou segundo
timeframes_padronizado = tuple(f.timeframes_mesma_unidade(timeframes))
# tamanho das batches de treinamento
batch_size = 1024#512
# set seed pros resultados não variarem
seed = np.random.randint(99999)
seed = 777
tf.keras.utils.set_random_seed(seed)
np.random.seed(seed)
# pega tamanho da imagem (input)
img_shape = (40, 100, 3)#f.image_shape()
# porcentagem dos dados que vou usar pra treino e teste
pct_imagens_teste = 0.1
pct_imagens_validacao = 0.1
# numero de modelos pro ensemble
n_modelos=5
# proporção pra diminuir as imagens no momento da criação 1 é imagem inteira
img_size = 1.0
# investimento inicial em dolares
investimento_inicial = 1000
# fee em %
fee = 0.1

# Prepara os dados

### Baixa e corrige dados

In [3]:
# baixa os dados e junta cada timeframe e junta em 1 csv por timeframe
# retorna None se arquivo processado já existe e timeframe se ele criou um arquivo do 0
lista_processados = []
for timeframe in timeframes:
    lista_processados.append(f.baixa_e_concatena(ticker= ticker, timeframe=timeframe, ano_inicial=2017))

BTCUSDT-1m já processado
BTCUSDT-5m já processado
BTCUSDT-15m já processado
BTCUSDT-30m já processado
BTCUSDT-1h já processado


In [4]:
# insere linhas sem dados e preenche com 0
# só corrige os timeframes que foram processados anteriormente
lista_p_correcao = [item for item in lista_processados if item != None]
f.corrige_arquivos(lista_p_correcao)

Nada corrigido


### Cria janelas, GAF's e salva imagens
Imagens ficam localizadas na pasta dados e são separadas por clissificação de compra ou venda

In [3]:
# cria uma lista de numpy arrays com os dados de cada timeframe.
# Eles estão organizados na mesma ordem que os timeframes
dfs_close = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Close"])[::-1][["Close", "Close time"]]).to_numpy() for timeframe in timeframes]
dfs_close = tuple(dfs_close)

# faz a mesma coisa pros dados de volume
dfs_volume = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Volume"])[::-1][["Volume", "Close time"]]).to_numpy() for timeframe in timeframes]
dfs_volume = tuple(dfs_volume)

#dfs_close[0], dfs_volume[0]

In [4]:
#pasta = "teste"
#linhas = range(1,10)

# transforma janelas em numpy arrays salvas em varios arquivos .tfrecords
def gera_tfrecords(linha, dfs_close, dfs_volume, lookback, quantis, pasta, img_size):

    # cria numpy array com precos de fechamento e pega a timestamp e decisao (compra ou venda)
    df_janela, decisao, timestamp = f.cria_janela(linha, dfs_close, lookback)

    # se decisao não é long nem short ou tem um buraco nos dados no meio da janela, pula a iteração
    if decisao == None or (0 in df_janela):
        return

    df_janela_volume = f.cria_janela(linha, dfs_volume, lookback)[0]

    # cria gaf images
    img_array = np.vstack((f.cria_gaf(df_janela.T, quantis, img_size), f.cria_gaf(df_janela_volume.T, quantis, img_size)))

    # converte pra valor entre 0 e 1
    img_array = (img_array+1)/2

    path = f'./Dados/TFRecords/{pasta}/{decisao}/{timestamp}.tfrecords'

    # trnasforma decisao em numero
    if decisao == "short1":
        decisao = 1
    elif decisao == "long0":
        decisao = 0

    writer = tf.io.TFRecordWriter(path, options="ZLIB")

    record_bytes = tf.train.Example(features=tf.train.Features(feature={
                #'timestamp' : tf.train.Feature(int64_list=tf.train.Int64List(value=[timestamp])),
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[decisao])),
                "image": tf.train.Feature(float_list=tf.train.FloatList(value=img_array.flatten()))
        })).SerializeToString()
    writer.write(record_bytes)
    writer.close()
    
    return

#gera_tfrecords(linhas, dfs_close, dfs_volume, lookback, quantis, pasta, img_size)

In [5]:
# separa treino, validacao e teste
ultima_linha = len(dfs_close[0])-(lookback*timeframes_padronizado[-1])

# linhas de teste são as primeiras x%, pois os dados mais recentes vem antes
# linhas de validação vem logo após
# linhas de treino são as restantes
linhas_teste = range(1, int(ultima_linha*pct_imagens_teste))
linhas_validacao = range(int(ultima_linha*pct_imagens_teste), int(ultima_linha*(pct_imagens_validacao+pct_imagens_teste)))
linhas_treino = range(int(ultima_linha*(pct_imagens_validacao+pct_imagens_teste)), ultima_linha+1)

print("qtd de linhas para teste", len(linhas_teste))
print("qtd de linhas para validacao", len(linhas_validacao))
print("qtd de linhas para treino", len(linhas_treino))

qtd de linhas para teste 282527
qtd de linhas para validacao 282528
qtd de linhas para treino 2260225


In [6]:
# gera arquivos .tfrecords

# gera dados de teste
for linha in tqdm(linhas_teste):
    gera_tfrecords(linha, dfs_close, dfs_volume, lookback, quantis, "teste", img_size)
# gera dados de validacao
for linha in tqdm(linhas_validacao):
    gera_tfrecords(linha, dfs_close, dfs_volume, lookback, quantis, "validacao", img_size)
# gera dados de treino
for linha in tqdm(linhas_treino):
    gera_tfrecords(linha, dfs_close, dfs_volume, lookback, quantis, "treino", img_size)

  0%|          | 0/282527 [00:00<?, ?it/s]

  0%|          | 0/282528 [00:00<?, ?it/s]

  0%|          | 0/2260225 [00:00<?, ?it/s]

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

...

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

## Cria dataset

In [13]:
# fazer lista de arquivo e label
#pasta = "teste"
#path = f"./Dados/Arrays/{pasta}/"
def pega_files_labels(path):
    list_subfolders = [f.name for f in os.scandir(path) if f.is_dir()]
    list_subfolders

    df = pd.DataFrame()
    
    i=0
    for subfolder in list_subfolders:
        path2 = path + str(subfolder)
        # pega arquivos .npz
        arquivos = [item.path for item in os.scandir(path2) if item.name.endswith(".tfrecords")]
        # poe num df temporario que iremos juntar com o principal
        df_temp = pd.DataFrame(arquivos, columns=["arquivos"])
        df_temp["labels"] = i
        # junta com o principal
        df = pd.concat([df, df_temp], axis=0, ignore_index=True)
        i += 1
    # retorna lista de arquivos e labels
    return df["arquivos"].to_numpy(), df["labels"].to_numpy(dtype=bool)

#filenames, labels = pega_files_labels(path)
#filenames, labels

In [22]:
def decode_fn(tfrecord, img_shape=img_shape):
    '''
    Decodifica arquivos .tfrecords para que seja acessado como um dicionário pelo tensorflow
    
    Args:
    tfrecord: Arquivo tfrecord lido com a função tf.data.TFRecordDataset
    img_shape: shape de numpy array
    '''
    return tf.io.parse_single_example(
      # Data
      tfrecord,

      # Schema
      {
        #'timestamp': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        "image": tf.io.FixedLenFeature(img_shape, tf.float32)},
    )

In [23]:
def pega_imagem(tfrecord, img_shape=img_shape):
    return decode_fn(tfrecord, img_shape)["image"]

def pega_imagem_e_label(tfrecord, img_shape=img_shape):
    decoded = decode_fn(tfrecord, img_shape)
    return decoded["image"], decoded["label"]

In [25]:
def cria_batches(batch_size=batch_size, valid_data=False, test_data=False, train_data=False):
    # se for treino precisa embaralhar antes
    if train_data == True:
        print("Criando dados de treino")
        pasta = "treino"
        path = f"./Dados/TFRecords/{pasta}/"
        # pega nomes e labels
        filenames, labels = pega_files_labels(path)
        # embaralha
        np.random.shuffle(filenames)
        # lê arquivos
        data = tf.data.TFRecordDataset(
            filenames, compression_type="ZLIB").map(
            pega_imagem_e_label, num_parallel_calls=tf.data.AUTOTUNE)
        # pega batch
        data_batch = data.batch(batch_size)
        return data_batch
    # se for validacao não embaralha
    elif valid_data == True:
        print("Criando dados de validacao")
        pasta = "validacao"
        path = f"./Dados/TFRecords/{pasta}/"
        # pega nomes e labels
        filenames, labels = pega_files_labels(path)
        # le a array
        data = tf.data.TFRecordDataset(
            filenames, compression_type="ZLIB").map(
            pega_imagem_e_label, num_parallel_calls=tf.data.AUTOTUNE)
        # pega batch
        data_batch = data.batch(batch_size)
        return data_batch
    # Se for teste não tem labels
    elif test_data == True:
        print("Criando dados de teste")
        pasta = "teste"
        path = f"./Dados/TFRecords/{pasta}/"
        # pega nomes e labels
        filenames, labels = pega_files_labels(path)
        # le a array
        data = tf.data.TFRecordDataset(
            filenames, compression_type="ZLIB").map(pega_imagem, num_parallel_calls=tf.data.AUTOTUNE)
        # pega batch
        data_batch = data.batch(batch_size)
        return data_batch
    else:
        print("Chame a função corretamente")
        return

In [26]:
#train_data = cria_batches(train_data=True)
#validation_data = cria_batches(valid_data=True)
test_data = cria_batches(test_data=True)

Criando dados de teste


E:\anaconda\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


---------------------------------------------
testes  e arquivo

In [ ]:
# agora tem que ver como faz pra ler e manter as dimensões originais
# precisa parsear
def parse_proto(example_proto):
    features = {
    'X': tf.FixedLenFeature((345,), tf.float32),
    'y': tf.FixedLenFeature((5,), tf.float32),
    }
    parsed_features = tf.parse_single_example(example_proto, features)
    return parsed_features['X'], parsed_features['y']

def read_tfrecords(file_names=("file1.tfrecord", "file2.tfrecord", "file3.tfrecord"),
                   buffer_size=10000,
                   batch_size=100):
    dataset = tf.contrib.data.TFRecordDataset(file_names)
    dataset = dataset.map(parse_proto)
    dataset = dataset.shuffle(buffer_size)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    return tf.contrib.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)

In [20]:
# como pegar timestamps e labels depois
data = tf.data.TFRecordDataset(f'./Dados/TFRecords/teste/teste.tfrecords',
                               compression_type="ZLIB").map(lambda x: decode_fn(x, img_shape))
lista = []
for item in data:
    lista_temp = [item["timestamp"].numpy(), item["label"].numpy()]
    lista.append(lista_temp)
pd.DataFrame(lista, columns = ["timestamp", "label"])

E:\anaconda\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


282290

In [ ]:
def cria_batches(batch_size=batch_size, valid_data=False, test_data=False, train_data=False):
    # se for treino precisa embaralhar antes
    if train_data == True:
        print("Criando dados de treino")
        pasta = "treino"
        # le a array
        data = tf.data.TFRecordDataset(f'./Dados/TFRecords/{pasta}/{pasta}.tfrecords',
                               compression_type="ZLIB").map(lambda x: pega_imagem_e_label(x, img_shape),
                                                            num_parallel_calls=tf.data.AUTOTUNE)
        # embaralha
        data = data.shuffle(buffer_size=len(data))
        # pega batch
        data_batch = data.batch(batch_size)
        return data_batch
    # se for validacao não embaralha
    elif valid_data == True:
        print("Criando dados de validacao")
        pasta = "validacao"
        path = f"./Dados/Arrays/{pasta}/"
        # pega nomes e labels
        filenames, labels = pega_files_labels(path)
        # coloca nome do arquivo e label no tensor
        data = tf.data.Dataset.from_tensor_slices((filenames, labels))
        # le a array
        data = data.map(lambda x, y: tf.numpy_function(
                  pega_array_e_label, [x, y], Tout=[tf.float32, tf.bool]), num_parallel_calls=tf.data.AUTOTUNE)#data.map(pega_array_e_label)
        # pega batch
        data_batch = data.batch(batch_size)
        return data_batch
    # Se for teste não tem labels
    elif test_data == True:
        print("Criando dados de teste")
        pasta = "teste"
        path = f"./Dados/Arrays/{pasta}/"
        # pega nomes e labels
        filenames, labels = pega_files_labels(path)
        # coloca nome do arquivo e label no tensor
        data = tf.data.Dataset.from_tensor_slices((filenames))
        # le a array
        data = data.map(lambda x: tf.numpy_function(
                  pega_array, [x], tf.float32), num_parallel_calls=tf.data.AUTOTUNE)#data.map(pega_array_e_label)
        # pega batch
        data_batch = data.batch(batch_size)
        return data_batch
    else:
        print("Chame a função corretamente")
        return

In [ ]:
def pega_imagem(tfrecord_decoded):
    return tfrecord_decoded["image"]

def pega_imagem_e_label(tfrecord_decoded):
    return tfrecord_decoded["image"], tfrecord_decoded["label"]

In [ ]:
# como pegar timestamps e labels depois
data = tf.data.TFRecordDataset(f'./Dados/TFRecords/teste/teste.tfrecords',
                               compression_type="ZLIB").map(lambda x: decode_fn(x, img_shape))
lista = []
for item in data:
    lista_temp = [item["timestamp"].numpy(), item["label"].numpy()]
    lista.append(lista_temp)
pd.DataFrame(lista, columns = ["timestamp", "label"])

# Old
gera 1 tfrecords por janela

In [4]:
#pasta = "teste"
#linhas = range(1,10)

# transforma janelas em numpy arrays salvas em um único arquivo .tfrecords
# isso é ruim pra dar shuffle depois
def gera_tfrecords(linhas, dfs_close, dfs_volume, lookback, quantis, pasta, img_size):

    path = f'./Dados/TFRecords/{pasta}/{pasta}.tfrecords'#.encode('utf-8')
    writer = tf.io.TFRecordWriter(path, options="ZLIB")
    for linha in tqdm(linhas):
        # cria numpy array com precos de fechamento e pega a timestamp e decisao (compra ou venda)
        df_janela, decisao, timestamp = f.cria_janela(linha, dfs_close, lookback)

        # se decisao não é long nem short ou tem um buraco nos dados no meio da janela, pula a iteração
        if decisao == None or (0 in df_janela):
            continue

        df_janela_volume = f.cria_janela(linha, dfs_volume, lookback)[0]

        # cria gaf images
        img_array = np.vstack((f.cria_gaf(df_janela.T, quantis, img_size), f.cria_gaf(df_janela_volume.T, quantis, img_size)))

        # converte pra valor entre 0 e 1
        img_array = (img_array+1)/2
        
        
        # trnasforma decisao em numero
        if decisao == "short1":
            decisao = 1
        elif decisao == "long0":
            decisao = 0
        
        
        record_bytes = tf.train.Example(features=tf.train.Features(feature={
                    'timestamp' : tf.train.Feature(int64_list=tf.train.Int64List(value=[timestamp])),
                    'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[decisao])),
                    "image": tf.train.Feature(float_list=tf.train.FloatList(value=img_array.flatten()))
            })).SerializeToString()
        writer.write(record_bytes)
    
    writer.close()
    img_shape = img_array.shape
    return img_shape

#gera_tfrecords(linhas, dfs_close, dfs_volume, lookback, quantis, pasta, img_size)

In [65]:
pasta = "teste"
linha = 99

# transforma janelas em numpy arrays salvas em arquivos .tfrecords separados por timeframe
def gera_tfrecords(linha, dfs_close, dfs_volume, lookback, quantis, pasta, img_size):
    # tentativa de criar um arquivo npz com todas as numpy arrays. muito lento
    df_janela, decisao, timestamp = f.cria_janela(linha, dfs_close, lookback)
    
    # se decisao não é long nem short ou tem um buraco nos dados no meio da janela, pula a iteração
    if decisao == None or (0 in df_janela):
        return
    
    df_janela_volume = f.cria_janela(linha, dfs_volume, lookback)[0]
    
    # cria gaf images
    img_array = np.vstack((f.cria_gaf(df_janela.T, quantis, img_size), f.cria_gaf(df_janela_volume.T, quantis, img_size)))
    
    # converte pra valor entre 0 e 1
    img_array = (img_array+1)/2
    
    # pega dimensoes da imagem
    #rows, cols, depth = img_array.shape

    path = f'./Dados/TFRecords/{pasta}/{decisao}/{timestamp}.tfrecords'#.encode('utf-8')
    
    if decisao == "short1":
        decisao = 1
    elif decisao == "long0":
        decisao = 0
    
    # Write the records to a file.
    with tf.io.TFRecordWriter(path, options="ZLIB") as file_writer:
        record_bytes = tf.train.Example(features=tf.train.Features(feature={
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[decisao])),
                "image": tf.train.Feature(float_list=tf.train.FloatList(value=(img_array.flatten())))
        })).SerializeToString()
        file_writer.write(record_bytes)
    
    return

gera_tfrecords(linha, dfs_close, dfs_volume, lookback, quantis, pasta, img_size)

In [68]:
def decode_fn(tfrecord, img_shape=img_shape):
    '''
    Decodifica arquivos .tfrecords para que seja acessado como um dicionário pelo tensorflow
    
    Args:
    tfrecord: Arquivo tfrecord lido com a função tf.data.TFRecordDataset
    img_shape: shape de numpy array
    '''
    return tf.io.parse_single_example(
      # Data
      tfrecord,

      # Schema
      {
        'label': tf.io.FixedLenFeature([], tf.int64),
        "image": tf.io.FixedLenFeature(img_shape, tf.float32)}
    )

# Old

Funciona gerando arquivos .npz, mas demora 20 min por epoch.

In [4]:
# gerando cada array em um arquivo comprimido individual
def gera_numpy_arrays(linha, dfs_close, dfs_volume, lookback, quantis, pasta, img_size):
    # tentativa de criar um arquivo npz com todas as numpy arrays. muito lento
    df_janela, decisao, timestamp = f.cria_janela(linha, dfs_close, lookback)
    
    # se decisao não é long nem short ou tem um buraco nos dados no meio da janela, pula a iteração
    if decisao == None or (0 in df_janela):
        return
    
    df_janela_volume = f.cria_janela(linha, dfs_volume, lookback)[0]
    
    # cria gaf images
    img_array = np.vstack((f.cria_gaf(df_janela.T, quantis, img_size), f.cria_gaf(df_janela_volume.T, quantis, img_size)))
    
    # converte pra valor entre 0 e 1
    img_array = (img_array+1)/2
    
    # salva como imagem na pasta (treino ou teste)/(long ou short)
    #np.save(f'./Dados/Arrays/{pasta}/{decisao}/{timestamp}', np.float32(img_array))
    np.savez_compressed(f'./Dados/Arrays/{pasta}/{decisao}/{timestamp}', np.float32(img_array))
    
    return

In [5]:
# separa treino, validacao e teste
ultima_linha = len(dfs_close[0])-(lookback*timeframes_padronizado[-1])

# linhas de teste são as primeiras x%, pois os dados mais recentes vem antes
# linhas de validação vem logo após
# linhas de treino são as restantes
linhas_teste = range(1, int(ultima_linha*pct_imagens_teste))
linhas_validacao = range(int(ultima_linha*pct_imagens_teste), int(ultima_linha*(pct_imagens_validacao+pct_imagens_teste)))
linhas_treino = range(int(ultima_linha*(pct_imagens_validacao+pct_imagens_teste)), ultima_linha+1)

print("qtd de linhas para teste", len(linhas_teste))
print("qtd de linhas para validacao", len(linhas_validacao))
print("qtd de linhas para treino", len(linhas_treino))

qtd de imagens para teste 282527
qtd de imagens para validacao 282528
qtd de imagens para treino 2260225


In [6]:
# gera arquivos npz
for linha in tqdm(linhas_teste):
    gera_numpy_arrays(linha, dfs_close, dfs_volume, lookback, quantis, "teste", img_size)
for linha in tqdm(linhas_validacao):
    gera_numpy_arrays(linha, dfs_close, dfs_volume, lookback, quantis, "validacao", img_size)
for linha in tqdm(linhas_treino):
    gera_numpy_arrays(linha, dfs_close, dfs_volume, lookback, quantis, "treino", img_size)

  0%|          | 0/282527 [00:00<?, ?it/s]

  0%|          | 0/282528 [00:00<?, ?it/s]

  0%|          | 0/2260225 [00:00<?, ?it/s]

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

...

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller for sample [0]. Consider decreasing the number of bins or removing these samples.
  warn("Some quantiles are equal. The number of bins will "
E:\anaconda\lib\site-packages\pyts\preprocessing\discretizer.py:168: UserWarning: Some quantiles are equal. The number of bins will be smaller fo

In [7]:
# testa
x = np.load(f'./Dados/Arrays/teste/long0/1672531139999.npz')
x.files

['arr_0']

## Cria dataset

Aparentemente consegui sem usar arquivos. Preciso testar

In [13]:
# fazer lista de arquivo e label
#pasta = "treino"
#path = f"./Dados/Arrays/{pasta}/"
def pega_files_labels(path):
    list_subfolders = [f.name for f in os.scandir(path) if f.is_dir()]
    list_subfolders

    df = pd.DataFrame()
    subfolder = list_subfolders[0]

    i=0
    for subfolder in list_subfolders:
        path2 = path + str(subfolder)
        # pega arquivos .npz
        arquivos = [item.path for item in os.scandir(path2) if item.name.endswith(".npz")]
        # poe num df temporario que iremos juntar com o principal
        df_temp = pd.DataFrame(arquivos, columns=["arquivos"])
        df_temp["labels"] = i
        # junta com o principal
        df = pd.concat([df, df_temp], axis=0, ignore_index=True)
        i += 1
    # retorna lista de arquivos e labels
    return df["arquivos"].to_numpy(), df["labels"].to_numpy(dtype=bool)

#filenames, labels = pega_files_labels(path)
#filenames, labels

In [10]:
def pega_array(path):
    return (np.load(path)["arr_0"])

def pega_array_e_label(path, label):
    array = pega_array(path)
    return (array, label)

In [11]:
def cria_batches(batch_size=batch_size, valid_data=False, test_data=False, train_data=False):
    # se for treino precisa embaralhar antes
    if train_data == True:
        print("Criando dados de treino")
        pasta = "treino"
        path = f"./Dados/Arrays/{pasta}/"
        # pega nomes e labels
        filenames, labels = pega_files_labels(path)
        # coloca nome do arquivo e label no tensor
        data = tf.data.Dataset.from_tensor_slices((filenames, labels))
        # embaralha antes de virar janela pq é mais rapido
        data = data.shuffle(buffer_size=len(data))
        # le a array
        data = data.map(lambda x, y: tf.numpy_function(
                  pega_array_e_label, [x, y], Tout=[tf.float32, tf.bool]), num_parallel_calls=tf.data.AUTOTUNE)#data.map(pega_array_e_label)
        # pega batch
        data_batch = data.batch(batch_size)
        return data_batch
    # se for validacao não embaralha
    elif valid_data == True:
        print("Criando dados de validacao")
        pasta = "validacao"
        path = f"./Dados/Arrays/{pasta}/"
        # pega nomes e labels
        filenames, labels = pega_files_labels(path)
        # coloca nome do arquivo e label no tensor
        data = tf.data.Dataset.from_tensor_slices((filenames, labels))
        # le a array
        data = data.map(lambda x, y: tf.numpy_function(
                  pega_array_e_label, [x, y], Tout=[tf.float32, tf.bool]), num_parallel_calls=tf.data.AUTOTUNE)#data.map(pega_array_e_label)
        # pega batch
        data_batch = data.batch(batch_size)
        return data_batch
    # Se for teste não tem labels
    elif test_data == True:
        print("Criando dados de teste")
        pasta = "teste"
        path = f"./Dados/Arrays/{pasta}/"
        # pega nomes e labels
        filenames, labels = pega_files_labels(path)
        # coloca nome do arquivo e label no tensor
        data = tf.data.Dataset.from_tensor_slices((filenames))
        # le a array
        data = data.map(lambda x: tf.numpy_function(
                  pega_array, [x], tf.float32), num_parallel_calls=tf.data.AUTOTUNE)#data.map(pega_array_e_label)
        # pega batch
        data_batch = data.batch(batch_size)
        return data_batch
    else:
        print("Chame a função corretamente")
        return

## Treino

In [14]:
train_data = cria_batches(train_data=True)
validation_data = cria_batches(valid_data=True)
test_data = cria_batches(test_data=True)

Criando dados de treino


E:\anaconda\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Criando dados de validacao


E:\anaconda\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Criando dados de teste


E:\anaconda\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [15]:
# rede CNN
def CNN(img_shape=img_shape):
    model = keras.Sequential()
    model.add(layers.Input(shape=img_shape))
    
    # reescala os dados pra entre 0 e 1
    #model.add(layers.Rescaling(1./255))
    
    # convolucao
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(1, 2)))
    model.add(layers.Dropout(0.15))
    
    # classificação
    model.add(layers.Flatten(name="flatten"))
    model.add(layers.Dense(16, activation="relu"))
    model.add(layers.Dense(8, activation="linear"))
    model.add(layers.Dense(1, activation="sigmoid"))
    
    opt = Adagrad(learning_rate=0.1)
    model.compile(loss = "binary_crossentropy", optimizer=opt, metrics=["accuracy",
                                                                     keras.metrics.Precision(),
                                                                     keras.metrics.Recall(), 
                                                                     keras.metrics.AUC()])

    return model

In [16]:
modelos = [CNN() for i in range(n_modelos)]
resultado_CNN = np.empty((n_modelos, 5))
i = 0
for model in modelos:
    print("Modelo:", i)
    # precisa definir o checkpoint antes de começar cada CNN. Se não acaba usando um do outro
    checkpoint = ModelCheckpoint(f"./modelos/modelo CNN {i}", monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
    # para de treinar se a acuracia de treino parar de aumentar por 3 epochs
    es = EarlyStopping(monitor='val_accuracy', patience=4, mode="max")
    callbacks_list = [checkpoint, es]

    # treina CNN hinge
    model.fit(train_data,  epochs=10, use_multiprocessing=True, callbacks=callbacks_list, validation_data=validation_data) #verbose=0
    model = keras.models.load_model(f"./modelos/modelo CNN {i}")
    resultado_CNN[i] = model.evaluate(validation_data)
    i = i + 1

Modelo: 0
Epoch 1/10
  61/2081 [..............................] - ETA: 20:18 - loss: 0.7675 - accuracy: 0.4980 - precision: 0.4986 - recall: 0.5767 - auc: 0.4993

KeyboardInterrupt: 

# old

In [6]:
#linha= tf.constant(99)
# gerando cada array em tensor. Não funciona pq buga operação com indexador
def gera_numpy_tensors(linha, dfs_close=dfs_close, dfs_volume=dfs_volume, lookback=lookback, quantis=quantis, img_size=img_size):
    #linha = linha.numpy()
    linha = int(linha)
    #print(type(linha2))
    # tentativa de criar um arquivo npz com todas as numpy arrays. muito lento
    df_janela, decisao, timestamp = f.cria_janela(linha, dfs_close, lookback)
    
    # se decisao não é long nem short ou tem um buraco nos dados no meio da janela, retorna um tensor de tamanho 0
    if decisao == None or (0 in df_janela):
        return (tf.zeros(0, dtype=tf.float32, name=timestamp), tf.zeros(0, dtype=tf.bool))
    
    df_janela_volume = f.cria_janela(linha, dfs_volume, lookback)[0]
    
    # cria gaf images
    img_array = np.vstack((f.cria_gaf(df_janela.T, quantis, img_size), f.cria_gaf(df_janela_volume.T, quantis, img_size)))
    
    if decisao == "short1":
        decisao = True
    elif decisao == "long0":
        decisao = False
    
    # converte pra valor entre 0 e 1
    img_array = (img_array+1)/2
    
    return (tf.convert_to_tensor(np.float32(img_array), name=timestamp), tf.convert_to_tensor(decisao, name=timestamp))
#gera_numpy_tensors(linha)

In [7]:
def gera_numpy_tensors_sem_label(linha, dfs_close=dfs_close, dfs_volume=dfs_volume, lookback=lookback, quantis=quantis, img_size=img_size):
    #linha = linha.numpy()
    linha = int(linha)
    #print(type(linha2))
    # tentativa de criar um arquivo npz com todas as numpy arrays. muito lento
    df_janela, decisao, timestamp = f.cria_janela(linha, dfs_close, lookback)
    
    # se decisao não é long nem short ou tem um buraco nos dados no meio da janela, retorna um tensor de tamanho 0
    if decisao == None or (0 in df_janela):
        return (tf.zeros(0, dtype=tf.float32, name=timestamp))
    
    df_janela_volume = f.cria_janela(linha, dfs_volume, lookback)[0]
    
    # cria gaf images
    img_array = np.vstack((f.cria_gaf(df_janela.T, quantis, img_size), f.cria_gaf(df_janela_volume.T, quantis, img_size)))
    
    if decisao == "short1":
        decisao = True
    elif decisao == "long0":
        decisao = False
    
    # converte pra valor entre 0 e 1
    img_array = (img_array+1)/2
    
    return (tf.convert_to_tensor(np.float32(img_array), name=timestamp))

In [8]:
def cria_batches(linhas, batch_size=batch_size, valid_data=False, test_data=False, train_data=False):
    # se for treino precisa embaralhar antes
    if train_data == True:
        print("Criando dados de treino")
        data = tf.data.Dataset.from_tensor_slices(tf.constant(np.array(linhas)))
        # embaralha antes de virar janela pq é mais rapido
        data = data.shuffle(buffer_size=len(linhas))
        # cria janelas e label
        data = data.map(lambda x: tf.py_function(func=gera_numpy_tensors,
                  inp=[x], Tout=[tf.float32, tf.bool]), deterministic=False)
        # joga fora se o tamanho do tensor resultante for 0
        data = data.filter(lambda x, y: tf.not_equal(tf.size(x), 0))
        # cria batch
        data_batch = data.batch(batch_size)
        return data_batch
    # se for validacao não embaralha
    elif valid_data == True:
        print("Criando dados de validacao")
        data = tf.data.Dataset.from_tensor_slices(tf.constant(np.array(linhas)))
        # cria janelas e label
        data = data.map(lambda x: tf.py_function(func=gera_numpy_tensors,
                  inp=[x], Tout=[tf.float32, tf.bool]), deterministic=False)
        # joga fora se o tamanho do tensor resultante for 0
        data = data.filter(lambda x, y: tf.not_equal(tf.size(x), 0))
        # cria batch
        data_batch = data.batch(batch_size)
        return data_batch
    # Se for teste não tem labels
    elif test_data == True:
        print("Criando dados de teste")
        data = tf.data.Dataset.from_tensor_slices(tf.constant(np.array(linhas)))
        # embaralha antes de virar janela pq é mais rapido
        data = data.shuffle(buffer_size=len(linhas))
        # cria janelas e label
        data = data.map(lambda x: tf.py_function(func=gera_numpy_tensors_sem_label,
                  inp=[x], Tout=[tf.float32]), deterministic=False)
        # joga fora se o tamanho do tensor resultante for 0
        data = data.filter(lambda x: tf.not_equal(tf.size(x), 0))
        # cria batch
        data_batch = data.batch(batch_size)
        return data_batch
    else:
        print("Chame a função corretamente")
        return

In [9]:
train_data = cria_batches(linhas_treino, train_data=True)
validation_data = cria_batches(linhas_validacao, valid_data=True)
test_data = cria_batches(linhas_teste, test_data=True)

Criando dados de treino
Criando dados de validacao
Criando dados de teste


E:\anaconda\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:2200: UserWarning: The `deterministic` argument has no effect unless the `num_parallel_calls` argument is specified.
  warnings.warn("The `deterministic` argument has no effect unless the "
E:\anaconda\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [10]:
# visualiza
train_data.element_spec, validation_data.element_spec, test_data.element_spec

((TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
  TensorSpec(shape=<unknown>, dtype=tf.bool, name=None)),
 (TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),
  TensorSpec(shape=<unknown>, dtype=tf.bool, name=None)),
 (TensorSpec(shape=<unknown>, dtype=tf.float32, name=None),))

In [5]:
ultima_linha = len(dfs_close[0])-(lookback*timeframes_padronizado[-1])
linhas_teste = range(1, int(ultima_linha*pct_imagens_teste))
linhas_validacao = range(int(ultima_linha*pct_imagens_teste), int(ultima_linha*(pct_imagens_validacao+pct_imagens_teste)))
linhas_treino = range(int(ultima_linha*(pct_imagens_validacao+pct_imagens_teste)), ultima_linha+1)

linhas_teste, linhas_validacao, linhas_treino

(range(1, 282528), range(282528, 565056), range(565056, 2825281))

In [19]:
# primeiro cria um dataset com as linhas que vamos usar
data = tf.data.Dataset.from_tensor_slices(tf.constant(np.array(linhas_teste)))
# depois passa a função pra gerar os tensors
data = data.map(gera_numpy_tensors)
#data = data.map(teste)
# joga fora os Nones
data = data.filter(lambda x: x != None)
data_batch = data.batch(BATCH_SIZE)

E:\anaconda\lib\site-packages\tensorflow\python\framework\tensor_util.py:455: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  nparray = np.array(values, dtype=np_dt)


TypeError: in user code:

    File "C:\Users\Guilherme\AppData\Local\Temp\ipykernel_10988\822893404.py", line 7, in gera_numpy_tensors  *
        df_janela, decisao, timestamp = tf.py_function(func=f.cria_janela, inp=[linha, dfs_close, lookback], Tout=tf.float64)

    TypeError: Tensors in list passed to 'input' of 'EagerPyFunc' Op have types [int32, <NOT CONVERTIBLE TO TENSOR>, int32] that are invalid. Tensors: [<tf.Tensor 'args_0:0' shape=() dtype=int32>, (array([[1.65424000e+04, 1.67253120e+12],
           [1.65364200e+04, 1.67253114e+12],
           [1.65377800e+04, 1.67253108e+12],
           ...,
           [4.28056000e+03, 1.50294258e+12],
           [4.26148000e+03, 1.50294252e+12],
           [4.26148000e+03, 1.50294246e+12]]), array([[1.6542400e+04, 1.6725312e+12],
           [1.6540420e+04, 1.6725309e+12],
           [1.6539050e+04, 1.6725306e+12],
           ...,
           [4.2614800e+03, 1.5029433e+12],
           [4.2614800e+03, 1.5029430e+12],
           [4.2614800e+03,

In [22]:
# cria arquivo npz vazio
np.savez_compressed(f'./Dados/Arrays/teste/long0/arrays')
np.savez_compressed(f'./Dados/Arrays/teste/short1/arrays')
np.savez_compressed(f'./Dados/Arrays/treino/long0/arrays')
np.savez_compressed(f'./Dados/Arrays/treino/short1/arrays')

def gera_numpy_arrays(linha, dfs_close, dfs_volume, lookback, quantis, pasta, img_size):
    # tentativa de criar um arquivo npz com todas as numpy arrays. muito lento
    df_janela, decisao, timestamp = f.cria_janela(linha, dfs_close, lookback)
    
    # se decisao não é long nem short ou tem um buraco nos dados no meio da janela, pula a iteração
    if decisao == None or (0 in df_janela):
        return
    
    df_janela_volume = f.cria_janela(linha, dfs_volume, lookback)[0]
    
    # cria gaf images
    img_array = np.vstack((f.cria_gaf(df_janela.T, quantis, img_size), f.cria_gaf(df_janela_volume.T, quantis, img_size)))
    
    # salva como imagem na pasta (treino ou teste)/(long ou short)
    
    #save_img(f'./Dados/Imagens/{pasta}/{decisao}/{timestamp}.png', array_to_img(img_array), scale=False)
    np.save(f'{timestamp}', img_array)
    z = zipfile.ZipFile(f'./Dados/Arrays/{pasta}/{decisao}/arrays.npz', "a",zipfile.ZIP_DEFLATED)
    z.write(f'{timestamp}.npy')
    z.close()
    os.remove(f'{timestamp}.npy')
    
    return

# Prova que o cria_janelas funciona perfeitamente

In [9]:
if __name__ == '__main__':
    # criação de DF gigante e cheio de nan's

    def une_timeframes(timeframes=timeframes):
        # junta todos os timeframes em um df só
        # esse é o jeito lento e serve apenas pra fazer uma prova real
        # pega o primeiro timeframe com as colunas desejadas
        df = pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframes[0]}.csv", index_col = "Close time"
                         , usecols=["Close time","Open", "High", "Low", "Close", "Volume", "Number of trades"])
        #df.index = pd.to_datetime(df.index, unit="ms")

        # coloca o sufixo nas colunas do menor timeframe
        df.columns = df.columns+f"_{timeframes[0]}"
        print(timeframes[0])

        # junta todos os outros timeframes e coloca o sufixo correto
        for timeframe in timeframes[1:]:
            print(timeframe)
            df2 = pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", index_col = "Close time",
                             usecols=["Close time","Open", "High", "Low", "Close", "Volume", "Number of trades"])
            df2.columns = df2.columns+f"_{timeframe}"
            df = df.merge(df2, how="left", on=["Close time"])#, suffixes=(None, f"_{timeframe}"))


        # transforma o index em datetime
        df.index = pd.to_datetime(df.index, unit="ms")

        #inverte a ordem do df pq vamos usar ele do fim pro começo pra fazer as janelas
        df.sort_index(axis=0, inplace = True, ascending=False)
        return(df)
    df = une_timeframes(timeframes)
    df[99:].head(20)

1m
5m
15m
30m
1h


In [10]:
df[99:].head(20)

,Open_1m,High_1m,Low_1m,Close_1m,Volume_1m,Number of trades_1m,Open_5m,High_5m,Low_5m,Close_5m,...,Low_30m,Close_30m,Volume_30m,Number of trades_30m,Open_1h,High_1h,Low_1h,Close_1h,Volume_1h,Number of trades_1h
Close time,,,,,,,,,,,,,,,,,,,,,
2022-12-31 22:20:59.999,16531.61,16537.39,16528.91,16532.26,110.79296,2708,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:19:59.999,16519.14,16531.70,16506.36,16531.62,638.64815,8301,16564.95,16565.62,16506.36,16531.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:18:59.999,16547.34,16547.85,16515.16,16519.14,408.04045,6108,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:17:59.999,16554.12,16554.23,16545.78,16547.34,96.98766,2665,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:16:59.999,16562.27,16563.55,16553.57,16554.12,109.60176,2619,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:15:59.999,16564.95,16565.62,16561.78,16562.73,76.53378,1809,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:14:59.999,16562.26,16567.49,16562.14,16565.38,98.09439,2583,16556.18,16567.49,16555.75,16565.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:13:59.999,16562.15,16563.80,16560.68,16562.26,70.03443,1721,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:12:59.999,16565.11,16565.16,16560.26,16561.81,77.08899,1988,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
colunas = [coluna for coluna in df.columns if "Close" in coluna]
df[colunas]

,Close_1m,Close_5m,Close_15m,Close_30m,Close_1h
0,16542.40,16542.40,16542.4,16542.4,16542.4
1,16536.42,NaN,NaN,NaN,NaN
2,16537.78,NaN,NaN,NaN,NaN
3,16540.52,NaN,NaN,NaN,NaN
4,16539.28,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2826475,4261.48,4261.48,NaN,NaN,NaN
2826476,4261.48,NaN,NaN,NaN,NaN
2826477,4280.56,NaN,NaN,NaN,NaN
2826478,4261.48,NaN,NaN,NaN,NaN


In [28]:
linha = 99
df2 = pd.DataFrame()
for coluna in colunas:
    df2[coluna] = df[coluna].iloc[linha:].dropna().head(lookback).reset_index(drop=True)
df2

,Close_1m,Close_5m,Close_15m,Close_30m,Close_1h
0,16532.26,16531.62,16565.38,16548.28,16548.28
1,16531.62,16565.38,16548.28,16563.84,16568.60
2,16519.14,16556.18,16560.53,16568.60,16570.14
3,16547.34,16543.94,16563.84,16570.71,16577.79
4,16554.12,16548.28,16566.31,16570.14,16582.08
5,16562.73,16549.59,16568.60,16572.17,16600.39
6,16565.38,16556.15,16570.46,16577.79,16590.10
7,16562.26,16560.53,16570.71,16577.13,16586.43
8,16561.81,16560.07,16569.76,16582.08,16600.89
9,16565.11,16557.92,16570.14,16583.43,16575.44


In [29]:
pd.DataFrame(f.cria_janela(linha, dfs_close, lookback)[0])

,0,1,2,3,4
0,16532.26,16531.62,16565.38,16548.28,16548.28
1,16531.62,16565.38,16548.28,16563.84,16568.60
2,16519.14,16556.18,16560.53,16568.60,16570.14
3,16547.34,16543.94,16563.84,16570.71,16577.79
4,16554.12,16548.28,16566.31,16570.14,16582.08
5,16562.73,16549.59,16568.60,16572.17,16600.39
6,16565.38,16556.15,16570.46,16577.79,16590.10
7,16562.26,16560.53,16570.71,16577.13,16586.43
8,16561.81,16560.07,16569.76,16582.08,16600.89
9,16565.11,16557.92,16570.14,16583.43,16575.44


In [31]:
# prova que o timestamp também está certo
f.cria_janela(linha, dfs_close, lookback)[2]

1672525259999

In [32]:
pd.to_datetime(f.cria_janela(linha, dfs_close, lookback)[2], unit="ms")

Timestamp('2022-12-31 22:20:59.999000')